# Solve

MAXVALUE = ...

$ money_{n, r, i} $ - Цена, по которой купил человек n, из региона r, товар i. $ money \in [0, MAXVALUE) $

$ buy_{n, i} $ - бинарная переменная. Купил ли человек n, товар i.

$ greater\_equal_{n, k} $ где $ k \in {0, 1000, 2000, ... 6000}$ - бинарная переменная. Клиент потратил больше или равно k денег

$ less\_equal_{n, k} $ где $ k \in {-1, 999, 1999, 2999, 3999, 4999, 5999}$ - бинарная переменная. Клиент потратил <= k денег

$ nat_{n, r} $ - бинарная переменная, относится ли клиент n, к региону r

$ n \in [0, 95)$

$ r \in [0, 3) $

$ i \in [0, 5) $

[comment]: # ( $ p \in [0, MAX\_PRICE + 1) $)


|n| * |r| * |i| = 1425

# Functions

Потраченные деньги

$ \sum_{n,r,i} money_{n, r, i} $

Потраченные деньги на каждый товар в каждом регионе

$\forall n, r \sum_{i} money_{n, r, i} $

# Relationships

Связь money и greater\_equal

$ \forall n, k \sum_{r, i} money_{n, r, i} >= greater\_equal_{n, k} * k $ 

Связь money и less\_equal

$ \forall n, k \sum_{r, i} money_{n, r, i} <= MAXVALUE  - (MAXVALUE - k) * less\_equal_{n, k} $ 

Таблица 1

$ table1\_n= [32, 38, 10, 8, 2, 2, 3]$

$ \forall t : table1\_n\_sum= \sum_{i = 0}^{ i < t} table1\_n_i$

Связь из таблицы 1

$ \forall_{index} \sum_{n} greater\_equal_{n, k_{index}} = 95 - table1\_n\_sum_{index}$ 

$ \forall_{index} \sum_{n} less\_equal_{n, k_{index}} = table1\_n\_sum_{index} $ 

$ \forall n, k : less\_equal_{n, k}  + greater\_equal_{n, k} >= 1 $

$ \forall n : \sum_{k} less\_equal_{n, k}  + greater\_equal_{n, k} = len_k $

$ \forall n, k : less\_equal_{n, k}  <= less\_equal_{n, nextk} $

$ \forall n, k : greater\_equal_{n, k}  >= greater\_equal_{n, nextk} $

Таблицы 2

$ seg_1 = [33.000, 36.000] $

$ seg_2 = [120.000, 125.000] $

$ seg_3 = [10.000, 13.000] $

$ table2\_n = [22, 60, 13] $

Связь из таблицы 2

$ \forall r  : \sum_{n, i} money_{n, r, i} \in seg_r $


Связь переменных money с регионами

$ \forall n, r \sum_{i} money_{n, r, i} <=  nat_{n, r} * MAXVALUE $

Ограничения nat

$ \forall n \sum_{r} nat_{n, r} <= 1 $

$ \forall r \sum_{n} nat_{n, r} = table2\_n_{r} $

Таблица 3




$ seg_1 = [4.000, 6.000] $

$ seg_2 = [28.000, 30.000] $

$ seg_3 = [105.000, 109.000] $

$ seg_4 =  [16.000, 18.000] $

$ seg_5 =  [10.000, 12.000] $

$ table3\_n = [77, 87, 95, 91, 51] $


Ограничения money по сумме в соответствии с товаром

$ \forall i \sum_{n, r} money_{n, r, i} \in seg_{r} $

Ограничения buy

$ \forall i \sum_{n} buy_{n, i} = table3\_n_{i} $

Связь money с buy

$ \forall n, i : \sum_{r} money_{n, r, i} <=  buy_{n, i} * MAXVALUE $

Итоговое количество переменных: ~ 3800

Итоговое количество уравнений: ~ _

# Result

Проект не увенчался успехом, так как алгоритм не останавливается за разумное время (час). Скорее всего, из-за большого количества логических переменных библиотека не может дать результат.

# Code

In [1]:
import time


In [2]:
start = time.time()
start

1558442393.567904

In [3]:
from pulp import *
import numpy as np
import random

In [4]:
MAXVALUE = 1e4

In [5]:
len_n = 95
len_r = 3
len_i = 5
len_k = 7
greater_equal_border = list(map(lambda x: x * 1000, range(0, len_k)))
less_equal_border = list(map(lambda x: x * 1000 - 1, range(0, len_k)))

In [6]:
greater_equal_border


[0, 1000, 2000, 3000, 4000, 5000, 6000]

In [7]:
less_equal_border

[-1, 999, 1999, 2999, 3999, 4999, 5999]

# Var

In [8]:
def money(*, n, r, i):
    return _money[n][r][i]

In [9]:
_money = np.ndarray((len_n, len_r, len_i), dtype=object)

for n in range(len_n):
    for r in range(len_r):
        for i in range(len_i):
            _money[n, r, i] = LpVariable("money_{}_{}_{}".format(n, r, i), 0, MAXVALUE, cat='Continuous')

In [10]:
def buy(*, n, i):
    return _buy[n][i]

In [11]:
_buy = np.ndarray((len_n, len_i), dtype=object)

for n in range(len_n):
    for i in range(len_i):
        _buy[n, i] = LpVariable("buy_{}_{}".format(n, i), cat='Binary')

In [12]:
def greater_equal(*, n, k):
    return _greater_equal[n][k]

In [13]:
_greater_equal = np.ndarray((len_n, len_k), dtype=object)

for n in range(len_n):
    for k in range(len_k):
        _greater_equal[n, k] = LpVariable("greater_equal_{}_{}".format(n, k),cat='Binary')

In [14]:
def less_equal(*, n, k):
    return _less_equal[n][k]

In [15]:
_less_equal = np.ndarray((len_n, len_k), dtype=object)

for n in range(len_n):
    for k in range(len_k):
        _less_equal[n, k] = LpVariable("less_equal_{}_{}".format(n, k),cat='Binary')

In [16]:
def nat(*, n, r):
    return _nat[n][r]

In [17]:
_nat = np.ndarray((len_n, len_r), dtype=object)

for n in range(len_n):
    for r in range(len_r):
        _nat[n, r] = LpVariable("nat_{}_{}".format(n, r),cat='Binary')

# Relationships

In [18]:
prob = LpProblem('task', LpMaximize)

prob += lpSum(_money)

In [19]:
for n in range(len_n):
    for k in range(len_k):
        prob += lpSum(list(money(n=n, r=r, i=i) \
                           for r in range(len_r) \
                           for i in range(len_i))) \
                >= greater_equal(n=n, k=k) * greater_equal_border[k]

In [20]:
for n in range(len_n):
    for k in range(len_k):
        prob += lpSum(list(money(n=n, r=r, i=i) \
                           for r in range(len_r) \
                           for i in range(len_i))) \
                <= MAXVALUE - \
                (MAXVALUE -less_equal_border[k]) \
                * less_equal(n=n, k=k)

In [21]:
table1_n = [32, 38, 10, 8, 2, 2, 3]

In [22]:
table1_n_sum = [sum(table1_n[0:i]) for i in range(len(table1_n) + 1)]
table1_n_sum

[0, 32, 70, 80, 88, 90, 92, 95]

In [23]:
for k in range(len_k):
    prob += lpSum([greater_equal(n=n, k=k) \
                   for n in range(len_n)]) \
            == 95 - table1_n_sum[k]

In [24]:
for k in range(len_k):
    prob += lpSum([less_equal(n=n, k=k) \
                   for n in range(len_n)]) \
            == table1_n_sum[k]

In [25]:
for n in range(len_n):
    prob += lpSum([less_equal(n=n, k=k) + greater_equal(n=n, k=k) \
                   for k in range(len_k)]) \
            == len_k

In [26]:
for n in range(len_n):
    for k in range(len_k - 1):
        prob += less_equal(n=n, k=k) <= less_equal(n=n, k=k+1) 

In [27]:
for n in range(len_n):
    for k in range(len_k - 1):
        prob += greater_equal(n=n, k=k) >= greater_equal(n=n, k=k+1) 

In [28]:
prob.solve()

KeyboardInterrupt: 

In [ ]:
print( "Status:", LpStatus[prob.status])

In [ ]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

In [ ]:
end = time.time()
end - start